In [46]:
import pandas as pd 
import json
import re
import numpy as np 

In [65]:
data = []
with open('.\\데이터\\Library\\reviews.json', 'r', encoding='UTF-8') as f:
   for line in f:
        line = json.dumps(line)
        data.append(json.loads(line))

In [66]:
item = pd.DataFrame(data)

In [276]:
temp = []
exception = []
for index,value in item.iterrows():
    first = value[0].find("'comment'")
    second = value[0].find("'user'")
    content = value[0][first+12:second-3].replace("\"","'")
    content = content.replace("\\",'')
    review = "\""+content+"\""
    change = value[0].replace("'","\"")
    word = change[:first+11]+review+change[second-2:]
    try:
        temp.append(json.loads(word))
    except:
        exception.append([index, word])

In [278]:
dataframe = pd.DataFrame(temp)

In [279]:
dataframe.head()

,work,flags,unixtime,stars,nhelpful,time,comment,user
0,3206242,[],1194393600,5.0,0,"Nov 7, 2007",This a great book for young readers to be intr...,van_stef
1,12198649,[],1333756800,5.0,0,"Apr 7, 2012",Help Wanted: Tales of On The Job Terror from E...,dwatson2
2,12533765,[],1352937600,NaN,0,"Nov 15, 2012","Magoon, K. (2012). Fire in the streets. New Yo...",edspicer
3,12981302,[],1364515200,4.0,0,"Mar 29, 2013","Well, I definitely liked this book better than...",amdrane2
4,5231009,[],1270944000,3.0,0,"Apr 11, 2010",It's a nice science-fiction thriller with some...,Lila_Gustavus


In [286]:
len(dataframe)

1706979

In [280]:
import pickle

In [285]:
trust_data = pd.read_table('./데이터/Library/edges.txt', sep='\t', header =None, names=['friend1','friend2'])

In [287]:
trust_data.head()

,friend1,friend2
0,Rodo anehan,NaN
1,Rodo sevilemar,NaN
2,Rodo dingsi,NaN
3,Rodo slash,NaN
4,RelaxedReader AnnRig,NaN


In [288]:
len(trust_data)

219790

## 전처리

In [3]:
import pickle

In [5]:
with open('./데이터/Library/library_review.pickle', 'rb') as f:
    review = pickle.load(f)

In [12]:
review.head()

,work,flags,unixtime,stars,nhelpful,time,comment,user
0,3206242,[],1194393600,5.0,0,"Nov 7, 2007",This a great book for young readers to be intr...,van_stef
1,12198649,[],1333756800,5.0,0,"Apr 7, 2012",Help Wanted: Tales of On The Job Terror from E...,dwatson2
2,12533765,[],1352937600,NaN,0,"Nov 15, 2012","Magoon, K. (2012). Fire in the streets. New Yo...",edspicer
3,12981302,[],1364515200,4.0,0,"Mar 29, 2013","Well, I definitely liked this book better than...",amdrane2
4,5231009,[],1270944000,3.0,0,"Apr 11, 2010",It's a nice science-fiction thriller with some...,Lila_Gustavus


In [10]:
trust_data = pd.read_table('./데이터/Library/edges.txt', sep=' ', header =None, names=['friend1','friend2']) 

## user에게 index만들고 저장

In [15]:
user_set = set()
user_set.update(trust_data['friend1'].unique())
user_set.update(trust_data['friend2'].unique())
user_set.update(review['user'].unique())

In [24]:
map_user = []
for i, u in enumerate(user_set):
    map_user.append([u, i])

In [25]:
map_user_df = pd.DataFrame(map_user, columns = ['org_id', 'remap_id'])

In [27]:
map_user_df.to_csv('./git/Librarything/user_list.txt', sep = '\t', index =False)

## item에게 번호 mapping

In [29]:
item_set = set(review['work'].unique())

In [33]:
print("Unique item의 수 : ",len(item_set))
print("전체 Observation의 수 : ",len(review))

Unique item의 수 :  506165
전체 Observation의 수 :  1706979


In [34]:
map_item = []
for index, i in enumerate(item_set):
    map_item.append([i, index])
map_item_df = pd.DataFrame(map_item, columns = ['org_id', 'remap_id'])

In [35]:
map_item_df.to_csv('./git/Librarything/item_list.txt', sep = '\t', index =False)


## interaction mapping

In [40]:
map_item_dict = dict(zip(map_item_df['org_id'], map_item_df['remap_id']))
map_user_dict = dict(zip(map_user_df['org_id'], map_user_df['remap_id']))

In [42]:
u_i_rating = review[['user','work']]

In [43]:
u_i_rating['user'] = u_i_rating['user'].map(lambda x: map_user_dict[x])
u_i_rating['work'] = u_i_rating['work'].map(lambda x: map_item_dict[x])

ipykernel_launcher:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [44]:
u_i_rating.head()

,user,work
0,9441,150807
1,31373,156011
2,2671,333453
3,107435,37299
4,62053,185338


In [47]:
u_i_rating = np.asarray(u_i_rating)

In [49]:
interaction = []
for user_id in sorted(map_user_dict.values()):
    match = u_i_rating[u_i_rating[:,0] == user_id]
    if len(match)>1:
        interaction.append([user_id]+list(match[:,1]))

In [50]:
with open('./git/Library/total_data.txt', 'w') as output:
    for line in interaction:
        output.write(" ".join(map(str,line)))
        output.write('\n')

FileNotFoundError: [Errno 2] No such file or directory: './git/Library/total_data.txt'